In [1]:
import numpy as np
import math
import csv

In [2]:
class Node:
    def __init__(self,attribute):
        self.attribute = attribute
        self.children = []
        self.answer = ""

In [3]:
def read_data(filename):
    with open(filename) as dataset:
        buffer = csv.reader(dataset)
        headers = next(buffer)
        data = []
        for row in buffer:
            data.append(row)  
        return (headers,data)

In [4]:
def subtables(data,col,delete):
    cd = [row[col] for row in data] #column values 
    ucav = list(set(cd)) #unique column attribute values
    ucav_data = {} #data corresponding to the unique values
    
    for av in ucav:
        ucav_data[av] = [row for row in data if row[col]==av]
        
    if delete:
        for row in data:
            row.pop(col)
    
    return ucav,ucav_data

In [5]:
def entropy(S):
    last = list(set(S))
    if len(last) == 1:
        return 0
    entropy = {a : sum([1 for i in S if i == a])/(len(S)*1.0) for a in last}
    
    total_entropy = 0
    for e in entropy.values():
        total_entropy += -1 * e * math.log(e,2)
    
    return total_entropy

In [6]:
def compute_gain(data,col):
    attr,attr_records = subtables(data,col,delete=False)
    
    data_size = len(data)
    entropies = [0] * len(attr)
    ratio = [0] * len(attr)
    
    total_entropy = entropy([row[-1] for row in data]) #entropy of entire dataset
    for x in range(len(attr)):
        ratio[x] = len(attr_records[attr[x]]) / (data_size*1.0)
        entropies[x] = entropy([a[-1] for a in attr_records[attr[x]]])
        total_entropy -= ratio[x]*entropies[x]
    
    return total_entropy

In [7]:
def create_tree(data,features):
    target_column = [row[-1] for row in data]
    if (len(set(target_column))) == 1: #if all the values are the same
        node = Node("")
        node.answer = target_column[0]
        return node
    na = len(data[0])-1 #number of attributes
    gains = [0] * na
    for col in range(na):
        gains[col] = compute_gain(data,col) #calculate gain for each attribute
    split=gains.index(max(gains))
    node = Node(features[split])
    f = features[:split] + features[split+1:]
    attr,attr_records = subtables(data,split,delete=True)
    for x in range(len(attr)):
        child = create_tree(attr_records[attr[x]],f)
        node.children.append((attr[x],child))
    return node

In [8]:
def print_tree(node,level):
    if node.answer!="":
        print("  "*level,node.answer)
        return
    
    print("  "*level,node.attribute)
    for value,n in node.children:
        print("  "*(level+1),value)
        print_tree(n,level+2)

In [9]:
def classify(node,x_test,features):
    if node.answer!="":
        print(node.answer)
        return
    pos=features.index(node.attribute)
    for value, n in node.children:
        if x_test[pos]==value:
            classify(n,x_test,features)

In [10]:
features,dataset=read_data("dataset/id3.csv")
node1=create_tree(dataset,features)

print("The decision tree for the dataset using ID3 algorithm is")
print_tree(node1,0)

features,testdata=read_data("dataset/id3_test.csv")

for xtest in testdata:
    print("The test instance:",xtest)
    print("The label for test instance:",end="   ")
    classify(node1,xtest,features)

The decision tree for the dataset using ID3 algorithm is
 Outlook
   rain
     Wind
       weak
         yes
       strong
         no
   overcast
     yes
   sunny
     Humidity
       normal
         yes
       high
         no
The test instance: ['rain', 'cool', 'normal', 'strong']
The label for test instance:   no
The test instance: ['sunny', 'mild', 'normal', 'strong']
The label for test instance:   yes
